# DIME study data analysis using machine learning

## Loading the data

The data is loaded using metagenomics MGnify API. Two collections are loaded, one which contains participant information and another that contains the biome expression information.

In [15]:
import urllib.request
import json

STUDY_ID = "MGYS00006076" # DIME study

analyses_url = f"https://www.ebi.ac.uk/metagenomics/api/v1/studies/{STUDY_ID}?include=analyses&format=json"
samples_url = f"https://www.ebi.ac.uk/metagenomics/api/v1/studies/{STUDY_ID}?include=samples&format=json"

with urllib.request.urlopen(analyses_url) as f:
    analyses = json.load(f)

with urllib.request.urlopen(samples_url) as f:
    samples = json.load(f)

The participant and biome expression information are stored under different ids, so they must be collated using the infromation from the MGnify data.

In [16]:
# Participant and expression information are available in two different locations, under different ids
# We use the samples and analyses data to join the two information sources.

biosample_descs = {b['id']: (b['attributes']['sample-name'].upper(), b['attributes']['biosample']) for b in samples['included']}

mg_ids = {
    biosample_descs[m['relationships']['sample']['data']['id']]: m['id'] for m in analyses['included']
}

We prepare two tables, one which collects the participant information and one which collects biome expression information. The biome expression information is collected usint the [Biosamples](https://www.ebi.ac.uk/biosamples/) API.

They are connected using the DIME study measurement point signifier, e.g., `03-MP`, where `03` refers to the participant number and `MP` refers to the midpoint measurement.

In [17]:
# The individual expression samples and participant information are loaded and collected in two tables

rows_expression = {}
rows_participant = {}

for (sample_name, bio_id), sample_id in mg_ids.items():

    with urllib.request.urlopen(f"https://www.ebi.ac.uk/metagenomics/api/v1/analyses/{sample_id}/taxonomy/ssu?format=json") as r:
        d = json.loads(r.read())
    
    rows_expression[sample_name] = {
            o['id'].replace('::', ':'): o['attributes']['count'] for o in d['data']
        }
    
    with urllib.request.urlopen(f"https://www.ebi.ac.uk/biosamples/samples/{bio_id}.json") as r:
        bio = json.loads(r.read())
        
    s = bio['characteristics']['host sex'][0]['text'][0].upper()
    a = int(bio['characteristics']['host age'][0]['text'])
    d = '_'.join(bio['characteristics']['host diet'][0]['text'].lower().split(' ')[:2])
    
    rows_participant[sample_name] = {
        'age': a,
        'gender': s,
        'sample_arm': d
    }

The tables are converted into `pandas` data frames to allow for easier cleaning and manipulation during dataset construction. Any missing values in the biome expression table are filled in with `0.0`.

In [35]:
import pandas as pd

participant_table = pd.DataFrame.from_dict(rows_participant, orient='index')
expression_table = pd.DataFrame.from_dict(rows_expression, orient='index').fillna(0.0)

### Data normalization

The expression data for each measurement point are divided by the total sum to obtain relative expressions.

In [36]:
expression_table = expression_table.div(expression_table.sum(axis=1), axis=0)

### Preview of the participant and expression tables

In [21]:
participant_table

,age,gender,sample_arm
19-V1,54,F,before_low
04-V4,37,M,after_high
20-V3,43,F,before_low
22-MP,61,M,midpoint
01-V2,46,F,after_low
...,...,...,...
05-V3,36,M,before_low
15-MP,58,F,midpoint
20-V4,43,F,after_low
07-MP,29,F,midpoint


In [22]:
expression_table

,Archaea:Euryarchaeota:Methanobacteria:Methanobacteriales:Methanobacteriaceae:Methanobrevibacter,Bacteria,Bacteria:Actinobacteria:Actinobacteria:Bifidobacteriales:Bifidobacteriaceae:Bifidobacterium,Bacteria:Actinobacteria:Actinobacteria:Propionibacteriales:Propionibacteriaceae:Granulicoccus,Bacteria:Actinobacteria:Actinobacteria:Streptomycetales:Streptomycetaceae:Streptomyces:Streptomyces_sp._NRRL_F-6677,Bacteria:Actinobacteria:Coriobacteriia:Coriobacteriales:Coriobacteriaceae,Bacteria:Actinobacteria:Coriobacteriia:Coriobacteriales:Coriobacteriaceae:Collinsella,Bacteria:Bacteroidetes,Bacteria:Bacteroidetes:Bacteroidia,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales,...,Eukaryota:Viridiplantae:Streptophyta:Magnoliopsida:Malvales:Malvaceae,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_salyersiae,Bacteria:Actinobacteria:Coriobacteriia:Eggerthellales:Eggerthellaceae:Adlercreutzia,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Holdemania:Holdemania_massiliensis,Bacteria:Firmicutes:Negativicutes:Veillonellales,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Catenibacterium:Catenibacterium_mitsuokai,Bacteria:Firmicutes:Clostridia:Clostridiales:Peptostreptococcaceae:Romboutsia:Romboutsia_sp._MT17,Bacteria:Actinobacteria:Actinobacteria:Micrococcales:Micrococcaceae:Arthrobacter,Bacteria:Firmicutes:Clostridia:Clostridiales:Lachnospiraceae:Fusicatenibacter:Fusicatenibacter_saccharivorans,Bacteria:Firmicutes:Clostridia:Clostridiales:Syntrophomonadaceae:Syntrophomonas
19-V1,0.004831,0.067633,0.009662,0.004831,0.024155,0.004831,0.004831,0.019324,0.009662,0.009662,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
04-V4,0.005464,0.071038,0.005464,0.000000,0.038251,0.000000,0.010929,0.000000,0.010929,0.010929,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
20-V3,0.008696,0.056522,0.004348,0.004348,0.017391,0.000000,0.000000,0.004348,0.008696,0.008696,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
01-V2,0.007246,0.086957,0.007246,0.000000,0.036232,0.007246,0.000000,0.000000,0.014493,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
04-V1,0.006897,0.041379,0.006897,0.000000,0.013793,0.006897,0.000000,0.000000,0.013793,0.006897,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
07-V4,0.000000,0.080645,0.008065,0.000000,0.040323,0.000000,0.008065,0.056452,0.016129,0.008065,...,0.0,0.0,0.0,0.0,0.0,0.0,0.008065,0.000000,0.000000,0.000000
05-V3,0.000000,0.104839,0.016129,0.000000,0.000000,0.000000,0.008065,0.048387,0.024194,0.008065,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.008065,0.000000,0.000000
15-MP,0.000000,0.081818,0.018182,0.000000,0.009091,0.009091,0.009091,0.000000,0.036364,0.018182,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
07-MP,0.000000,0.058824,0.008403,0.000000,0.050420,0.000000,0.016807,0.016807,0.016807,0.016807,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


## Dataset preparation

To perform machine learning-based analysis on the DIME study data, we prepared multiple variants of the dataset. 

### Participant+ExpressionEndpoints
In this variant of the dataset we combined the participant information (age and gender) with the biome expression data of the taxonomy endpoints, i.e., entries that contain exactly 7 taxonomical levels. For the final feature we take the feature **sample_arm**, which describes at which measurement point the Expression measurements were taken as a discrete variable, with values of `before_low`, `after_low`, `midpoint`, `before_high` and `after_high`. Each data example corresponds to a particular participant and measurement point, resulting in a dataset with 100 examples.

In [33]:
# Prepare the Participant+ExpressionEndpoints dataset

species_endpoints = expression_table.loc[:,[len(tax.split(':')) == 7 for tax in expression_table.columns]].copy() # select only columns that contain exactly 7 taxonomical levels, i.e., they are endpoints

df_ends = participant_table.join(species_endpoints)

A preview of the Participant+ExpressionEndpoints dataset:

In [41]:
df_ends

,age,gender,sample_arm,Bacteria:Actinobacteria:Actinobacteria:Streptomycetales:Streptomycetaceae:Streptomyces:Streptomyces_sp._NRRL_F-6677,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Porphyromonadaceae:Gabonibacter:Gabonibacter_massiliensis,Bacteria:Firmicutes:Bacilli:Lactobacillales:Streptococcaceae:Streptococcus:Streptococcus_pyogenes,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Merdibacter:Merdibacter_massiliensis,Bacteria:Proteobacteria:Deltaproteobacteria:Desulfovibrionales:Desulfovibrionaceae:Mailhella:Mailhella_massiliensis,Bacteria:Verrucomicrobia:Verrucomicrobiae:Verrucomicrobiales:Akkermansiaceae:Akkermansia:Akkermansia_muciniphila,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_coprocola,...,Bacteria:Firmicutes:Bacilli:Lactobacillales:Streptococcaceae:Lactococcus:Lactococcus_lactis,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Erysipelatoclostridium:Erysipelatoclostridium_sp._SNUG30386,Bacteria:Fusobacteria:Fusobacteriia:Fusobacteriales:Leptotrichiaceae:Sneathia:Sneathia_amnii,Bacteria:Firmicutes:Clostridia:Clostridiales:Lachnospiraceae:Roseburia:Roseburia_sp._499,Bacteria:Firmicutes:Clostridia:Clostridiales:Ruminococcaceae:Negativibacillus:Negativibacillus_massiliensis,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_salyersiae,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Holdemania:Holdemania_massiliensis,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Catenibacterium:Catenibacterium_mitsuokai,Bacteria:Firmicutes:Clostridia:Clostridiales:Peptostreptococcaceae:Romboutsia:Romboutsia_sp._MT17,Bacteria:Firmicutes:Clostridia:Clostridiales:Lachnospiraceae:Fusicatenibacter:Fusicatenibacter_saccharivorans
19-V1,54,F,before_low,0.024155,0.004831,0.004831,0.004831,0.004831,0.004831,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
04-V4,37,M,after_high,0.038251,0.000000,0.000000,0.000000,0.000000,0.000000,0.005464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
20-V3,43,F,before_low,0.017391,0.000000,0.004348,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
22-MP,61,M,midpoint,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
01-V2,46,F,after_low,0.036232,0.000000,0.014493,0.000000,0.000000,0.007246,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05-V3,36,M,before_low,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
15-MP,58,F,midpoint,0.009091,0.000000,0.009091,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
20-V4,43,F,after_low,0.027523,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
07-MP,29,F,midpoint,0.050420,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


### Participant+ExpressionEndpoints+ExpressionAggregates

This variant of the dataset expands on the one described above, by adding aggregates over all taxonomic levels, such as total values for `Archaea`, `Archaea:Methanobacteriota`, etc. As before this dataset consists of 100 examples.

In [37]:
# Prepare the Participant+ExpressionEndpoints+ExpressionAggregates dataset

import warnings
warnings.filterwarnings("ignore")

df_agg = df_ends.copy()

df_agg["TOTAL"] = 0


for name in expression_table:
    taxonomy = name.replace('?', '').strip(':').split(':')
    if taxonomy != ['']:
        path = ''
        for tax in taxonomy:
            path += ";" + tax
            path = path.strip(':')
            if path not in df_agg:
                df_agg[path] = 0
            if path != name: # do not override existing endpoints
                df_agg[path] += expression_table[name]
    df_agg['TOTAL'] += expression_table[name]

df_agg = df_agg.copy()

A preview of the Participant+ExpressionEndpoints+ExpressionAggregates dataset:

In [39]:
df_agg

,age,gender,sample_arm,Bacteria:Actinobacteria:Actinobacteria:Streptomycetales:Streptomycetaceae:Streptomyces:Streptomyces_sp._NRRL_F-6677,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Porphyromonadaceae:Gabonibacter:Gabonibacter_massiliensis,Bacteria:Firmicutes:Bacilli:Lactobacillales:Streptococcaceae:Streptococcus:Streptococcus_pyogenes,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Merdibacter:Merdibacter_massiliensis,Bacteria:Proteobacteria:Deltaproteobacteria:Desulfovibrionales:Desulfovibrionaceae:Mailhella:Mailhella_massiliensis,Bacteria:Verrucomicrobia:Verrucomicrobiae:Verrucomicrobiales:Akkermansiaceae:Akkermansia:Akkermansia_muciniphila,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_coprocola,...,;Bacteria;Bacteroidetes;Bacteroidia;Bacteroidales;Bacteroidaceae;Bacteroides;Bacteroides_salyersiae,;Bacteria;Actinobacteria;Coriobacteriia;Eggerthellales;Eggerthellaceae;Adlercreutzia,;Bacteria;Firmicutes;Erysipelotrichia;Erysipelotrichales;Erysipelotrichaceae;Holdemania;Holdemania_massiliensis,;Bacteria;Firmicutes;Erysipelotrichia;Erysipelotrichales;Erysipelotrichaceae;Catenibacterium;Catenibacterium_mitsuokai,;Bacteria;Firmicutes;Clostridia;Clostridiales;Peptostreptococcaceae;Romboutsia;Romboutsia_sp._MT17,;Bacteria;Actinobacteria;Actinobacteria;Micrococcales;Micrococcaceae,;Bacteria;Actinobacteria;Actinobacteria;Micrococcales;Micrococcaceae;Arthrobacter,;Bacteria;Firmicutes;Clostridia;Clostridiales;Lachnospiraceae;Fusicatenibacter;Fusicatenibacter_saccharivorans,;Bacteria;Firmicutes;Clostridia;Clostridiales;Syntrophomonadaceae,;Bacteria;Firmicutes;Clostridia;Clostridiales;Syntrophomonadaceae;Syntrophomonas
19-V1,54,F,before_low,0.024155,0.004831,0.004831,0.004831,0.004831,0.004831,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
04-V4,37,M,after_high,0.038251,0.000000,0.000000,0.000000,0.000000,0.000000,0.005464,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
20-V3,43,F,before_low,0.017391,0.000000,0.004348,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
22-MP,61,M,midpoint,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
01-V2,46,F,after_low,0.036232,0.000000,0.014493,0.000000,0.000000,0.007246,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05-V3,36,M,before_low,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.008065,0.008065,0.000000,0.000000,0.000000
15-MP,58,F,midpoint,0.009091,0.000000,0.009091,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
20-V4,43,F,after_low,0.027523,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
07-MP,29,F,midpoint,0.050420,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


### Participant+ExpressionDifference

In this variant, we combined participant information (age and gender) with the difference in the Expression measurements after a food treatment was done, i.e., the difference between measurement points **after_low** and **before_low** as well as the difference between after_high and before_high. The final feature encodes which dietary treatment was done, i.e., **high** and **low**. As all midpoint data is dropped, and the remaining before and after data points are paired up, the dataset consists of 40 examples.

In [25]:
# Prepare the Participant+ExpressionDifference dataset

df = df_ends.copy()

df["participant"] = df.index.astype(str)

df["participant"] = df["participant"].apply(lambda p: p.split('-')[0])

patient_features = ['age', 'gender']

def diff(d):
    return d[1] - d[0]

maps = {k: diff for k in df_ends.columns if k != "sample_arm"}
maps.update(
    {f: "first" for f in patient_features}
)

df_diff_low = df[(df["sample_arm"] == "before_low") | (df["sample_arm"] == "after_low")].groupby(by="participant").agg(
    maps
)

df_diff_high = df[(df["sample_arm"] == "before_high") | (df["sample_arm"] == "after_high")].groupby(by="participant").agg(
    maps
)


df_diff_low["diet"] = "low"
df_diff_low["p"] = df_diff_low.index.astype(str) + "-low"
df_diff_low.set_index("p", inplace=True)

df_diff_high["diet"] = "high"
df_diff_high["p"] = df_diff_high.index.astype(str) + "-high"
df_diff_high.set_index("p", inplace=True)

df_diff = pd.concat([df_diff_low, df_diff_high])

A preview of the Participant+ExpressionEndpoints+ExpressionAggregates dataset:

In [42]:
df_diff

,age,gender,Bacteria:Actinobacteria:Actinobacteria:Streptomycetales:Streptomycetaceae:Streptomyces:Streptomyces_sp._NRRL_F-6677,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Porphyromonadaceae:Gabonibacter:Gabonibacter_massiliensis,Bacteria:Firmicutes:Bacilli:Lactobacillales:Streptococcaceae:Streptococcus:Streptococcus_pyogenes,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Merdibacter:Merdibacter_massiliensis,Bacteria:Proteobacteria:Deltaproteobacteria:Desulfovibrionales:Desulfovibrionaceae:Mailhella:Mailhella_massiliensis,Bacteria:Verrucomicrobia:Verrucomicrobiae:Verrucomicrobiales:Akkermansiaceae:Akkermansia:Akkermansia_muciniphila,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_coprocola,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Prevotellaceae:Massiliprevotella:Massiliprevotella_massiliensis,...,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Erysipelatoclostridium:Erysipelatoclostridium_sp._SNUG30386,Bacteria:Fusobacteria:Fusobacteriia:Fusobacteriales:Leptotrichiaceae:Sneathia:Sneathia_amnii,Bacteria:Firmicutes:Clostridia:Clostridiales:Lachnospiraceae:Roseburia:Roseburia_sp._499,Bacteria:Firmicutes:Clostridia:Clostridiales:Ruminococcaceae:Negativibacillus:Negativibacillus_massiliensis,Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_salyersiae,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Holdemania:Holdemania_massiliensis,Bacteria:Firmicutes:Erysipelotrichia:Erysipelotrichales:Erysipelotrichaceae:Catenibacterium:Catenibacterium_mitsuokai,Bacteria:Firmicutes:Clostridia:Clostridiales:Peptostreptococcaceae:Romboutsia:Romboutsia_sp._MT17,Bacteria:Firmicutes:Clostridia:Clostridiales:Lachnospiraceae:Fusicatenibacter:Fusicatenibacter_saccharivorans,diet
p,,,,,,,,,,,,,,,,,,,,,
01-low,46,F,-0.020359,0.000000,0.009317,0.000000,0.000000,-0.007246,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low
02-low,44,M,0.011029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low
04-low,37,M,0.015619,0.000000,-0.007911,0.000000,0.000000,0.000000,0.000000,-0.006897,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low
05-low,36,M,-0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low
06-low,29,F,0.043431,0.000000,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low
07-low,29,F,-0.013731,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008065,0.000000,low
09-low,32,F,0.037166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low
13-low,37,M,0.018556,0.000000,0.000000,0.000000,0.000000,0.000000,0.011236,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low
14-low,43,F,0.029579,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,low


## Machine learning analysis

We use the [Orange](https://orangedatamining.com/) data mining framework to perform the analysis. Before applying machine learning methods to the prepared datasets, we need to transform them into a compatible format.

In [26]:
# Convert datasets into Orange tables

from Orange.data.pandas_compat import table_from_frame
from Orange.data.domain import Domain
from Orange.data.table import Table

def convert_to_orange_table(data_frame, target_variable):
    table = table_from_frame(data_frame, force_nominal=False)
    target_idx = None
    for i, a in enumerate(table.domain.attributes):
        if a.name == target_variable:
            target_idx = i
            break
    descriptives = [a for i, a in enumerate(table.domain.attributes) if i != target_idx]
    domain = Domain(descriptives, table.domain.attributes[target_idx])
    return Table.from_table(domain, table)

table_ends = convert_to_orange_table(df_ends, 'sample_arm')
table_agg = convert_to_orange_table(df_agg, 'sample_arm')
table_diff = convert_to_orange_table(df_diff, 'diet')

The prepared datasets are collected into a list along with their descriptors.

In [27]:
DATASETS = [("Participant+ExpressionEndpoints", table_ends), ("Participant+ExpressionEndpoints+ExpressionAggregates", table_agg), ("Participant+ExpressionDifference", table_diff)]

### Decision tree

We learn decision trees on all three of the datasets and print them out. The obtained models are presented in only in textual form. By examining the structure and hierarchy of the tree, one can determine which features are more influential in making predictions. In particular, the features closer to the root affect more data and are thus prescribed a higher influence, while features appearing deeper into the tree affect smaller portions of data and are thus deemed less influential.

In [28]:
from Orange.classification import TreeLearner

import Orange

def print_tree(node, level=0):
    """String representation of tree for debug purposees"""
    res = ""
    for child in node.children:
        if child:
            res += (f"{'    ' * level}{node.attr.name} {child.description}: {child.value if type(child) == Orange.tree.Node else ''}\n")
            res += print_tree(child, level + 1)
    return res

for table_name, table in DATASETS:
    model = TreeLearner(min_samples_leaf=5).fit_storage(table)

    print(f"Decision tree for {table_name}")

    print(print_tree(model.root))
    
    print("------------------------------------------------------------------------------------")

Decision tree for Participant+ExpressionEndpoints
Bacteria:Firmicutes:Clostridia:Clostridiales:Lachnospiraceae:Roseburia:Roseburia_hominis ≤ 0: 
    Bacteria:Firmicutes:Bacilli:Lactobacillales:Streptococcaceae:Streptococcus:Streptococcus_pyogenes ≤ 0.00917431: 
        Bacteria:Actinobacteria:Actinobacteria:Streptomycetales:Streptomycetaceae:Streptomyces:Streptomyces_sp._NRRL_F-6677 ≤ 0.00909091: [0. 0. 2. 1. 4.]
        Bacteria:Actinobacteria:Actinobacteria:Streptomycetales:Streptomycetaceae:Streptomyces:Streptomyces_sp._NRRL_F-6677 > 0.00909091: 
            Bacteria:Firmicutes:Bacilli:Lactobacillales:Streptococcaceae:Streptococcus:Streptococcus_pyogenes ≤ 0.00588235: 
                Bacteria:Firmicutes:Clostridia:Clostridiales:Ruminococcaceae:Ruminococcus:Ruminococcus_bromii ≤ 0: 
                    Bacteria:Firmicutes:Bacilli:Lactobacillales:Streptococcaceae:Streptococcus:Streptococcus_pyogenes ≤ 0.00483092: 
                        Bacteria:Proteobacteria:Betaproteobacteria:Bur

### Feature ranking

Feature ranking quantifies the above process, by assigning a numerical importance score to each of the descriptive (expression) features. We calculated rankings based on several approaches, i.e., Information Gain Ratio, Gini Decrease and ReliefF.

In [29]:
from Orange.preprocess.score import GainRatio, Gini, ReliefF

gain_ratio = GainRatio()
gini = Gini()
relieff = ReliefF()

#table_agg._compute_contingency()

RANKERS = [("Information Gain Ratio", gain_ratio), ("Gini Decrease", gini), ("ReliefF", relieff)]

for table_name, table in DATASETS:
    print("------------------------------------------------------------------------------------")
    for ranker_name, ranker in RANKERS:
        scores = ranker(table_ends)
        
        print(f"Top 10 features for {table_name} using {ranker_name}:")
        for i, (score, attr) in enumerate(sorted(zip(scores, table_ends.domain.attributes), key=lambda x: x[0], reverse=True)[:10]):
            print(f"{i+1:2d}\t{score:.5f}\t{attr.name}")
        print()


------------------------------------------------------------------------------------
Top 10 features for Participant+ExpressionEndpoints using Information Gain Ratio:
 1	0.33683	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_nordii
 2	0.31074	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_coprocola
 3	0.31074	Bacteria:Firmicutes:Clostridia:Clostridiales:Oscillospiraceae:Marseillibacter:Marseillibacter_massiliensis
 4	0.29510	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_massiliensis
 5	0.29289	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Rikenellaceae:Alistipes:Alistipes_shahii
 6	0.29289	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Rikenellaceae:Alistipes:Alistipes_finegoldii
 7	0.29230	Bacteria:Firmicutes:Clostridia:Clostridiales:Lachnospiraceae:Roseburia:Roseburia_hominis
 8	0.29104	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Porphyromonadaceae:Gab

Top 10 features for Participant+ExpressionEndpoints+ExpressionAggregates using Information Gain Ratio:
 1	0.33683	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_nordii
 2	0.31074	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_coprocola
 3	0.31074	Bacteria:Firmicutes:Clostridia:Clostridiales:Oscillospiraceae:Marseillibacter:Marseillibacter_massiliensis
 4	0.29510	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Bacteroidaceae:Bacteroides:Bacteroides_massiliensis
 5	0.29289	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Rikenellaceae:Alistipes:Alistipes_shahii
 6	0.29289	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Rikenellaceae:Alistipes:Alistipes_finegoldii
 7	0.29230	Bacteria:Firmicutes:Clostridia:Clostridiales:Lachnospiraceae:Roseburia:Roseburia_hominis
 8	0.29104	Bacteria:Bacteroidetes:Bacteroidia:Bacteroidales:Porphyromonadaceae:Gabonibacter:Gabonibacter_massiliensis
 9	0.29104	Bacteria:Firmicut